## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Bratsk,RU,56.1325,101.6142,63.09,82,100,1.25,overcast clouds
1,1,Albany,US,42.6001,-73.9662,92.05,43,0,9.66,clear sky
2,2,Barrow,US,71.2906,-156.7887,39.22,87,100,10.36,mist
3,3,Hailar,CN,49.2000,119.7000,63.97,84,23,8.12,few clouds
4,4,Tautira,PF,-17.7333,-149.1500,75.40,80,24,9.24,light rain
5,5,Todos Santos,MX,23.4500,-110.2167,104.31,79,80,8.30,broken clouds
6,6,Dusti,TJ,37.3481,68.6640,91.08,11,0,2.51,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,77.22,76,1,14.74,clear sky
8,8,Yellowknife,CA,62.4560,-114.3525,65.01,49,90,9.53,overcast clouds
9,9,Dryden,CA,49.7833,-92.7503,73.17,46,100,10.63,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = int(input("What is your maximum desired temperature for vaction?"))
min_temp = int(input("What is your minimum desired temperature for vacation?"))

What is your maximum desired temperature for vaction?100
What is your minimum desired temperature for vacation?85


In [ ]:
city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
isMaxTemp = city_data_df["Max Temp"] <= max_temp
isMinTemp = city_data_df["Max Temp"] >= min_temp
preferred_cities_df = city_data_df.loc[isMaxTemp & isMinTemp].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
1,1,Albany,US,42.6001,-73.9662,92.05,43,0,9.66,clear sky
6,6,Dusti,TJ,37.3481,68.6640,91.08,11,0,2.51,clear sky
11,11,Palafrugell,ES,41.9174,3.1631,88.56,73,94,5.01,overcast clouds
17,17,Jamestown,US,42.0970,-79.2353,85.01,57,20,13.80,few clouds
27,27,Rudbar,AF,30.1500,62.6000,98.42,11,19,6.04,few clouds
38,38,Buraydah,SA,26.3260,43.9750,98.64,10,19,10.36,few clouds
61,61,Sur,OM,22.5667,59.5289,88.02,66,100,11.23,overcast clouds
69,69,Itaituba,BR,-4.2761,-55.9836,89.37,49,99,4.50,overcast clouds
75,75,Altamira,BR,-3.2033,-52.2064,93.31,35,37,2.55,scattered clouds
76,76,Atar,MR,20.5169,-13.0499,86.56,59,63,18.01,light rain


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
# preferred_cities_df.isnull().sum()

City_ID            117
City               117
Country            117
Lat                117
Lng                117
Max Temp           117
Humidity           117
Cloudiness         117
Wind Speed         117
Current Weather    117
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Albany,US,92.05,clear sky,42.6001,-73.9662,
6,Dusti,TJ,91.08,clear sky,37.3481,68.6640,
11,Palafrugell,ES,88.56,overcast clouds,41.9174,3.1631,
17,Jamestown,US,85.01,few clouds,42.0970,-79.2353,
27,Rudbar,AF,98.42,few clouds,30.1500,62.6000,
38,Buraydah,SA,98.64,few clouds,26.3260,43.9750,
61,Sur,OM,88.02,overcast clouds,22.5667,59.5289,
69,Itaituba,BR,89.37,overcast clouds,-4.2761,-55.9836,
75,Altamira,BR,93.31,scattered clouds,-3.2033,-52.2064,
76,Atar,MR,86.56,light rain,20.5169,-13.0499,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Albany,US,92.05,clear sky,42.6001,-73.9662,
6,Dusti,TJ,91.08,clear sky,37.3481,68.6640,Hotel Izzat
11,Palafrugell,ES,88.56,overcast clouds,41.9174,3.1631,Hotel La Torre
17,Jamestown,US,85.01,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
27,Rudbar,AF,98.42,few clouds,30.1500,62.6000,
...,...,...,...,...,...,...,...
670,Miro Khan,PK,90.43,overcast clouds,27.7599,68.0921,Mukhtiar Dayo
673,The Valley,AI,87.66,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
680,Sao Joao Do Piaui,BR,91.36,overcast clouds,-8.3581,-42.2467,Nayra Palace Hotel
683,Sabancuy,MX,92.68,few clouds,18.9667,-91.1833,pulpo


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# vacation_df = pd.read_csv("WeatherPy_vacation.csv")
# vacation_df

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))